Combining Tree

In [15]:
%%writefile CombiningTree.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Collections;
using System.Threading;

namespace CombiningTree
{
    class CombiningTree
    {
        Node[] leaf;
        Node[] nodes;

        public CombiningTree(int width)
        {
            nodes = new Node[2 * width - 1];
            nodes[0] = new Node();
            for (int i = 1; i < nodes.Length; i++)
            {
                nodes[i] = new Node(nodes[(i - 1) / 2]);
            }
            leaf = new Node[width];
            for (int i = 0; i < leaf.Length; i++)
            {
                leaf[i] = nodes[nodes.Length - i - 1];
            }
        }

        static long Id()
        {
            return Thread.CurrentThread.ManagedThreadId;
        }

        public int getAndIncrement(int id)
        {
            Stack<Node> stack = new Stack<Node>();
            Node myLeaf = leaf[id % leaf.Length];
            Node node = myLeaf;
            // precombining phase
            try
            {
                while (node.precombine())
                {
                    node = node.parent;
                }
            }
            catch (Exception e) { Console.WriteLine(e); Console.ReadKey(); }
            Node stop = node;
            // combining phase
            int combined = 1;
            for (node = myLeaf; node != stop; node = node.parent)
            {
                try { combined = node.combine(combined); } catch (Exception) { }
                stack.Push(node);
            }


            // operation phase
            int prior = stop.op(combined);
            // distribution phase
            while (stack.Count > 0)
            {
                node = stack.Pop();
                node.distribute(prior);
            }
            return prior;


        }

        public int get()
        {
            return nodes[0].result;
        }
    }
}


Writing CombiningTree.cs


In [16]:
%%writefile Node.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Diagnostics;
using System.Threading;

namespace CombiningTree
{
	class Node
	{
		enum CStatus { IDLE, FIRST, SECOND, RESULT, ROOT };
		bool locked;
		CStatus cStatus;
		int firstValue, secondValue;
		public int result;
		public Node parent;

		public Node()
		{
			cStatus = CStatus.ROOT;
			locked = false;
		}
		public Node(Node myParent)
		{
			parent = myParent;
			cStatus = CStatus.IDLE;
			locked = false;
		}

		public bool precombine()
		{
			lock (this)
			{
				while (locked) Monitor.Wait(this);

				switch (cStatus)
				{
					case CStatus.IDLE:
						cStatus = CStatus.FIRST;
						return true;
					case CStatus.FIRST:
						locked = true;
						cStatus = CStatus.SECOND;
						return false;
					case CStatus.ROOT:
						return false;
					default:
						throw new Exception("unexpected Node state" + cStatus);
				}
			}
		}

		public int combine(int combined)
		{
			lock (this)
			{
				while (locked) Monitor.Wait(this);

				locked = true;
				firstValue = combined;
				switch (cStatus)
				{
					case CStatus.FIRST:
						return firstValue;
					case CStatus.SECOND:
						return firstValue + secondValue;
					default:
						throw new Exception("unexpected Node state " + cStatus);
				}
			}
		}

		public int op(int combined)
		{
			lock (this)
			{
				switch (cStatus)
				{
					case CStatus.ROOT:
						int prior = result;
						result += combined;
						return prior;
					case CStatus.SECOND:
						secondValue = combined;
						locked = false;

						Monitor.PulseAll(this); // wake up waiting threads

						while (cStatus != CStatus.RESULT) Monitor.Wait(this);
						locked = false;
						Monitor.PulseAll(this);

						cStatus = CStatus.IDLE;
						return result;
					default:
						throw new Exception("unexpected Node state");
				}
			}

		}

		public void distribute(int prior)
		{
			lock (this)
			{
				switch (cStatus)
				{
					case CStatus.FIRST:
						cStatus = CStatus.IDLE;
						locked = false;
						break;
					case CStatus.SECOND:
						result = prior + firstValue;
						cStatus = CStatus.RESULT;
						break;
					default:
						throw new Exception("unexpected Node state");
				}
				Monitor.PulseAll(this);
			}


		}
	}
}


Writing Node.cs


In [101]:
%%writefile TestCB.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;
using System.Diagnostics;
using System.IO;

namespace CombiningTree
{
    class TestCB
    {
        static CombiningTree tree;
        //static int TH = 12, NUM = 100;
        static int TH, NUM;
        static int width;
        // tree: combining tree where threads do increment ops
        // TH: number of threads
        // NUM: number of increment ops each thread performs

        static void Main(String[] args)
        {
            TH = int.Parse(args[0]);
            NUM = int.Parse(args[1]);
            
            width = (int) Math.Ceiling((decimal)TH / 2);
            tree = new CombiningTree(width);
            var stopWatch = new Stopwatch();


            Log(width + "-leaves Combining tree.");
            Log("Starting " + TH + " threads doing increments ...");

            stopWatch.Start();

            Parallel.For(0, TH, (index) =>
            {   
                DateTime start = DateTime.Now;
                for (int i = 0; i < NUM; i++)
                   tree.getAndIncrement(index);
                DateTime stop = DateTime.Now;
                Log(index + ": done in " + (stop - start).TotalSeconds + "s");
            });

            stopWatch.Stop();
            //var ts = stopWatch.Elapsed;
            var ts = stopWatch.Elapsed.TotalSeconds;
            Log("Total: " + tree.get());
            //Log("Total time: " + String.Format("{0:00}.{1:00}", ts.Seconds, ts.Milliseconds /10 )+'s');
            Console.WriteLine("Total time: " + "{0} ms", ts*1000);
            //Console.ReadKey();
            
            Writefile(ts*1000);
            
        }

        static void Log(String x)
        {
            Console.WriteLine(x);
        }
        
        public static async void Writefile(double ts) {
            using StreamWriter file = new("c-combining.txt", append: true);
            await file.WriteLineAsync("c#,"+TH+","+NUM+","+ts);
        }

}
}



Overwriting TestCB.cs


In [102]:
!csc TestCB.cs Node.cs CombiningTree.cs

Microsoft (R) Visual C# Compiler version 3.9.0-6.21160.10 (59eedc33)
Copyright (C) Microsoft Corporation. All rights reserved.



In [104]:
!TestCB.exe 12 1000

6-leaves Combining tree.
Starting 12 threads doing increments ...
4: done in 0.0189982s
10: done in 0.0200002s
5: done in 0.0200002s
11: done in 0.0200002s
3: done in 0.0329981s
9: done in 0.0349979s
6: done in 0.0349979s
8: done in 0.0349979s
2: done in 0.0369978s
7: done in 0.0359977s
0: done in 0.0379975s
1: done in 0.0369978s
Total: 12000
Total time: 43.5841 ms


Counting Network

In [145]:
%%writefile Balancer.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using System.Threading;

namespace BitonicCountingNetwork
{
    public class Balancer
    {
        private int toggle = 1;

        public int Traverse()
        {
            while(true){
                if (1 == Interlocked.Exchange(ref toggle, 0))
                {
                    return 0;
                }
                if (0 == Interlocked.Exchange(ref toggle, 1))
                {
                    return 1;
                }
            }
        }
    }
}


Overwriting Balancer.cs


In [128]:
%%writefile Bitonic.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace BitonicCountingNetwork
{
    public class Bitonic
    {
        private Bitonic[] half;

        private Merger merger;

        private readonly int width;

        public Bitonic(int wd)
        {
            width = wd;

            merger = new Merger(width);

            if (width > 2)
            {
                half = new Bitonic[]
                {
                    new Bitonic(width / 2),
                    new Bitonic(width / 2)
                };
            }
        }

        public int Traverse(int input)
        {
            int output = 0;

            if (width > 2)
            {
                output = half[input / (width / 2)].Traverse(input / 2);
            }
            return merger.Traverse((input >= (width / 2) ? (width / 2) : 0) + output);
        }
    }
}


Overwriting Bitonic.cs


In [130]:
%%writefile Merger.cs
using System;
using System.Collections.Generic;
using System.Linq;
using System.Text;
using System.Threading.Tasks;

namespace BitonicCountingNetwork
{
    class Merger
    {
        private Merger[] half;

        private Balancer[] layer;

        private readonly int width;

        public Merger(int wd)
        {
            width = wd;

            layer = new Balancer[width / 2];
            for (var i = 0; i < width / 2; ++i)
            {
                layer[i] = new Balancer();
            }

            if (width > 2)
            {
                half = new Merger[]
                {
                    new Merger(width / 2),
                    new Merger(width / 2)
                };
            }
        }

        public int Traverse(int input)
        {
            var output = 0;

            if (width <= 2) return layer[0].Traverse();

            if (input < width / 2)
            {
                output = half[input % 2].Traverse(input / 2);
            }
            else
            {
                output = half[1 - (input % 2)].Traverse(input / 2);
            }

            return (2 * output) + layer[output].Traverse();
        }
    }
}


Overwriting Merger.cs


In [146]:
%%writefile TestCN.cs
using System;
using System.Threading.Tasks;
using System.Diagnostics;
using System.Threading;
using System.IO;

namespace BitonicCountingNetwork
{
    class TestCN
    {
        static void Main(string[] args)
        {
            //const int width = 4;
            int width = int.Parse(args[0]);
            var bitonic = new Bitonic(width);
            var counters = new int[width];

            //const int tokenCount = 1000;
            int tokenCount = int.Parse(args[1]);
            var tokens = new int[tokenCount];

            var rand = new Random();
            var randLock = new object();
            var stopWatch = new Stopwatch();

            Parallel.For(0, tokenCount, (i) =>
            {
                int next;
                lock (randLock)
                {
                    next = rand.Next(width);
                }
                tokens[i] = next;
            });

            stopWatch.Start();
            Parallel.For(0, tokenCount, (i) =>
            {
                Interlocked.Increment(ref counters[bitonic.Traverse(tokens[i])]);
            });
            stopWatch.Stop();

            var traversing = stopWatch.Elapsed.TotalSeconds;
            //var traversing = stopWatch.ElapsedMilliseconds;

            for (var i = 0; i < width; ++i)
            {
                Console.WriteLine($"Output: {i} Count: {counters[i]}");
            }

            Console.WriteLine($"Time to traverse the network: {(traversing*1000).ToString()} ms");

            //Console.ReadKey();
            
            Writefile(traversing*1000, width, tokenCount);
        }
        
        public static async void Writefile(double ts, int width, int tokenCount) {
            using StreamWriter file = new("c-counting.txt", append: true);
            await file.WriteLineAsync($"c#,{width},{tokenCount},{ts}\n");
        }
    }
}


Overwriting TestCN.cs


In [147]:
!csc TestCN.cs Bitonic.cs Merger.cs Balancer.cs

Microsoft (R) Visual C# Compiler version 3.9.0-6.21160.10 (59eedc33)
Copyright (C) Microsoft Corporation. All rights reserved.



In [155]:
!TestCN.exe 4 1000000

Output: 0 Count: 250000
Output: 1 Count: 250000
Output: 2 Count: 250000
Output: 3 Count: 250000
Time to traverse the network: 85.9267 ms
